# Recuperando embeddings com o pacote [Minicons](https://github.com/kanishkamisra/minicons)

In [ ]:
!pip install minicons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:


# Vamos começar a usar modelos disponíveis no [Hugging Face](https://huggingface.co/). O modelo abaixo é um BERT treinado em português, o [BERTimbau](https://huggingface.co/neuralmind/bert-base-portuguese-cased)

# Mesma palavra, diferentes embeddings: conseguem entender a razão?

In [ ]:
from minicons import cwe

model = cwe.CWE('neuralmind/bert-base-portuguese-cased')

context_words = [("Eu peguei o banco.", "banco"),
                 ("E o banco era amarelo.", "banco")]

t1, t2 = model.extract_representation(context_words) # extraindo os vetores da palavra 'banco' da última camada do modelo
print(t1.shape, t2.shape)

print(model.extract_representation(context_words)) # veja que diferem abaixo




# Calculando as similaridades. Primeiro entre a mesma palavra em duas frases

In [ ]:
import torch.nn as nn


In [ ]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [ ]:

print(' similaridade entre eles: ',cos(t1, t2))


# Similaridade entre duas palavras distintas. Observe o valor de similaridade

In [ ]:
context_words = [("Eu sentei no banco.", "banco"),
                 ("Aquele banco que sentei quebrou.", "sentei")]

t1, t2 = model.extract_representation(context_words)

print(t1.shape, t2.shape)

print(model.extract_representation(context_words, layer = 12))
print(' similaridade entre eles: ',cos(t1, t2))


# Verificando score de contexto de uma palavra igual com sentido distinto (valor maior, menos relevante)

In [ ]:
from minicons import scorer
mlm_model = scorer.MaskedLMScorer('neuralmind/bert-base-portuguese-cased')

sentence = ['sentei no banco que estava no banco do brasil.']


mlm_model.token_score(sentence, PLL_metric='within_word_l2r')



# Verificando embeddings de sentenças. O Traansformer é orientado a tokens, uma das formas de recuperar a representaçào da sentença é fazendo a média entre as representações de cada token.

In [ ]:
!pip install -U sentence-transformers

# Similaridade entre duas sentenças que estão com as palavras em ordem distintas. Veja que o modelo usado abaixo tem uma saída menor que o BERT base (768) no estado escondido da última camada (384)

In [ ]:
sentences = ["hoje bem cedo fui ao banco e sentei no banco", "ao banco bem cedo fui hoje e no banco sentei"]


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print('Tamanho dos embeddings: ', embeddings.shape)
print(cos(torch.tensor(embeddings[0]), torch.tensor(embeddings[1])))

In [ ]:
!pip install transformers

# Agora a similaridade com o BERTimbau, fazendo a média explícita dos tokens

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:

# Carregando modelo e tokenizador explicitamente do HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:



# Mean Pooling - Take attention mask into account for correct averaging
# attention_mask: define os tokens válidos e os que foram adicionados por padding
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #O primeiro elemento contém  os embeddings de cada token
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()  #para garantir que a máscara tem o mesmo tamanho do token
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)





# Tokeniza as sentenças e deixa as duas no mesmo tamanho, para serem processadas juntas
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')


# Computar embeddings dos tokens
with torch.no_grad():
    model_output = model(**encoded_input)

# executar max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

print('Tamanho dos embeddings: ', sentence_embeddings.shape)
print('Similaridade: ', cos(torch.tensor(sentence_embeddings[0]), torch.tensor(sentence_embeddings[1])))


# Agora vamos ver a similaridade entre os embeddings de uma mesma palavra com sentido diferente em uma mesma frase, para a saída de cada bloco do Transfomer (BERT base tem 12 camadas)

In [ ]:
from transformers import BertConfig
config = BertConfig.from_pretrained('neuralmind/bert-base-portuguese-cased', output_hidden_states=True)
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased', config=config)

sentence = "hoje bem cedo fui ao banco pegar dinheiro e sentei no banco da praça"

# Tokeniza a primeira sentença apenas
encoded_input = tokenizer(sentence, return_tensors='pt')

tokens = tokenizer.tokenize(sentence, padding=True, truncation=True, return_tensors='pt')

#['hoje', 'bem', 'cedo', 'fu', '##i', 'ao', 'banco', 'pegar', 'dinheiro', 'e', 'sente', '##i', 'no', 'banco', 'da', 'praça']
# a primeira palavra banco é o sexto token e a segunda é o 13

pos1 = tokens.index('banco')
pos2 = tokens.index('banco', pos1+1)

print(pos1, tokens[pos1], pos2, tokens[pos2])






In [ ]:

# Computar token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# a saída terá a dimensão das camadas e dos tokens
for i in range(len(model_output.hidden_states)):
  t1 = model_output.hidden_states[i][0][pos1]
  t2 = model_output.hidden_states[i][0][pos2]
  print(f"A similaridade entre as palavras na camada {i} é {cos(t1,t2)}")